In [17]:
import pandas as pd
import numpy as np

In [2]:
major_conferences = pd.read_csv('major_conferences_filtered.csv')

In [24]:
major_conferences['Tagged Pitch Type'].value_counts(normalize = True)

Fastball       0.595905
Slider         0.177146
ChangeUp       0.101822
Curveball      0.089949
Cutter         0.013144
Undefined      0.009771
Sinker         0.007320
Splitter       0.002772
Other          0.002166
KnuckleBall    0.000005
Name: Tagged Pitch Type, dtype: float64

In [25]:
fastball_df = major_conferences[major_conferences['Tagged Pitch Type'] == 'Fastball']
slider_df = major_conferences[major_conferences['Tagged Pitch Type'] == 'Slider']
curveball_df = major_conferences[major_conferences['Tagged Pitch Type'] == 'Curveball']
changeup_df = major_conferences[major_conferences['Tagged Pitch Type'] == 'ChangeUp']

In [26]:
df_dict = {'fastball' : fastball_df,
          'slider': slider_df,
          'curveball' : curveball_df,
          'changeup' : changeup_df}

In [27]:
averages_dict = {}
for key in df_dict.keys():
    cur_df = df_dict[key]
    numeric_cols = []
    for col in cur_df.columns:
        if cur_df[col].dtype == float or cur_df[col].dtype == int:
            numeric_cols.append(col)
    unique_pitchers = cur_df['Pitcher Id'].unique()
    average_df = pd.DataFrame(index = unique_pitchers, columns = numeric_cols)
    
    for pitcher in average_df.index:
        pitcher_df = cur_df[cur_df['Pitcher Id'] == pitcher]
        for col in average_df.columns:
                average_df.loc[pitcher, col] = cur_df[col].mean()
    averages_dict[key+'_averages'] = average_df

In [28]:
def get_gb(hit_type):
    if hit_type == 'Undefined':
        return np.NaN
    elif hit_type == 'GroundBall':
        return 1
    else:
        return 0

In [29]:
def whiff(pitch_call):
    if pitch_call == 'StrikeSwinging':
        return 1
    else:
        return 0

In [30]:
for key in df_dict.keys():
    df_dict[key]['whiff%'] = df_dict[key]['Pitch Call'].apply(whiff)
    df_dict[key]['gb%'] = df_dict[key]['Hit Type'].apply(get_gb)
    for col in averages_dict[key+'_averages'].columns:
        cur_col = col + '_resid'
        df_dict[key][cur_col] = [np.NaN]*df_dict[key].index.size
    for i in df_dict[key].index:
        cur_pitcher = df_dict[key]['Pitcher Id'][i]
        for col in averages_dict[key+'_averages'].columns:
            cur_col = col + '_resid'
            df_dict[key][cur_col][i] = df_dict[key][col][i] - averages_dict[key + '_averages'][col][cur_pitcher]     

/Users/AngelaBeckham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/AngelaBeckham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/AngelaBeckham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [31]:
for key in df_dict.keys():
    df_dict[key].to_csv(key + '.csv')
for key in averages_dict.keys():
    averages_dict[key].to_csv(key + '.csv')

In [ ]:
for key in df_dict.keys():
    df_dict[key].to_csv(key+'.csv', )